In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.device("cpu")

device(type='cpu')

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 6)

    def forward(self, x):
        x = self.fc1(F.relu(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


In [4]:
data = [
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (4, 5),
    (5, 0),
    (0, 1),
    (0, 1),
    (5, 0),
    (0, 0),
    (0, 0),
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
]
data = torch.tensor(data)

In [5]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [9]:
num_epochs = 15

training_data = data[:, 0].reshape(-1, 1).float()
targets = data[:, 1].long()

for epoch in range(num_epochs):
    outputs = model(training_data)

    # Compute the loss
    loss = criterion(outputs,targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/15], Loss: 1.8025559186935425
Epoch [2/15], Loss: 1.800750970840454
Epoch [3/15], Loss: 1.798933982849121
Epoch [4/15], Loss: 1.7971326112747192
Epoch [5/15], Loss: 1.7953746318817139
Epoch [6/15], Loss: 1.7936865091323853
Epoch [7/15], Loss: 1.792089819908142
Epoch [8/15], Loss: 1.790602207183838
Epoch [9/15], Loss: 1.7892343997955322
Epoch [10/15], Loss: 1.7879905700683594
Epoch [11/15], Loss: 1.7868698835372925
Epoch [12/15], Loss: 1.7858672142028809
Epoch [13/15], Loss: 1.7849739789962769
Epoch [14/15], Loss: 1.7841796875
Epoch [15/15], Loss: 1.7834731340408325


In [10]:
model.eval()

Net(
  (fc1): Linear(in_features=1, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

In [11]:
model(torch.tensor([5.0]).reshape(-1, 1))

tensor([[0.0132, 0.0089, 0.0894, 0.3658, 0.4615, 0.0613]],
       grad_fn=<SoftmaxBackward0>)

In [15]:
training_data[:1, :].shape

torch.Size([1, 1])

In [19]:
torch.onnx.export(
    model,
    training_data[:1, :],
    "nn.onnx",
    export_params=True,
    opset_version=10,
    input_names = ['input'],
    output_names = ['output'],
)

In [17]:
import onnx

onnx_model = onnx.load("nn.onnx")
onnx.checker.check_model(onnx_model)

In [18]:
onnx_model

ir_version: 5
opset_import {
  version: 10
}
producer_name: "pytorch"
producer_version: "2.1.1"
graph {
  node {
    input: "input"
    output: "/Relu_output_0"
    name: "/Relu"
    op_type: "Relu"
  }
  node {
    input: "/Relu_output_0"
    input: "fc1.weight"
    input: "fc1.bias"
    output: "/fc1/Gemm_output_0"
    name: "/fc1/Gemm"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      type: FLOAT
      f: 1
    }
    attribute {
      name: "beta"
      type: FLOAT
      f: 1
    }
    attribute {
      name: "transB"
      type: INT
      i: 1
    }
  }
  node {
    input: "/fc1/Gemm_output_0"
    input: "fc2.weight"
    input: "fc2.bias"
    output: "/fc2/Gemm_output_0"
    name: "/fc2/Gemm"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      type: FLOAT
      f: 1
    }
    attribute {
      name: "beta"
      type: FLOAT
      f: 1
    }
    attribute {
      name: "transB"
      type: INT
      i: 1
    }
  }
  node {
    input: "/fc2/Gemm_output_0"
    ou